# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models

from collections import OrderedDict

import json
import time

from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

# Define transforms for the training, validation, and testing sets
# Data augmentation for the training set
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
# No data augmentation for validation or test datasets, just resizing and center+crop (+ of course same normalization)
test_valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_valid_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=test_valid_transforms)

# Using the image datasets and the trainforms, define the dataloaders
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# Use this cell to get a specific model and display its architecture (useful to get the input_features for the classifier)
model = models.vgg16(pretrained=True)
model

In [ ]:
def extend_pretrained_network(model, in_features):
    """
    Extend an existing and pretrained network by replacing its classifier only
    Note that model parameters are frozen so there will not be backprop through them
    """
    for param in model.parameters():
        param.requires_grad = False

    # Build a new classifier that will replace the existing one in the pretrained model
    # Take care of the size for input on this classifier, it might change depending on the chosen model
    # Note: as seen in the course, we are going to use the log-softmax as output which is a log probability 
    #       Using the log probability, computations are often faster and more accurate. To get the class probabilities
    #       later, we will need to take the exponential (torch.exp) of the output
    classifier = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(in_features, 500)),
                              ('relu', nn.ReLU()),
                              ('fc2', nn.Linear(500, 102)),
                              ('output', nn.LogSoftmax(dim=1))
                              ]))
    # Change the model's classifier: careful ! depending on the network, the name of the classifier might change...
    model.classifier = classifier

    return model


def build_model(model_name):
    # Define a dict where key is a model name and value is the number of features the classifier takes as input
    model_clf_inputs = {
        'resnet50': 2048,
        'densenet121': 1024,
        'vgg16': 25088
    }

    # Hard-coded cases, goal is not to have something more beautiful
    model = None
    if model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
    elif model_name == 'densenet121':
        model = models.densenet121(pretrained=True)

    in_features = model_clf_inputs[model_name]
    model = extend_pretrained_network(model, in_features)
    
    return model

In [ ]:
# Implement a function for the validation pass
def get_metrics(device, model, dataloader, criterion=None):
    '''
    Validation function for the validation pass on the validation dataloader
    '''
    loss = 0
    correct = 0
    total = 0
    model.eval()  # Make sure network is in eval mode for inference

    # Turn off gradients for validation --> saves memory and computations
    with torch.no_grad():
        for images, labels in dataloader:
            # Move input and label tensors to the right device
            images, labels = images.to(device), labels.to(device)

            output = model.forward(images)
            if criterion:
                loss += criterion(output, labels).item()

            total += labels.size(0)  # = batch size

            ps = torch.exp(output)  # Remember, the output is a log softmax ! So take the exp to get back the original

            # ps has shape(batch_size, nb_classes) so we take the index for which the computed value is the max among all classes
            # probabilities and we compare this to the ground truth which is labels.data
            # it then gives us a tensor of boolean that we can sum over to get the number of correctly classified images
            equality = (labels.data == ps.max(dim=1)[1])

            # Sum the number of correctly classified images among the given dataset
            correct += equality.type(torch.FloatTensor).sum().item()

    accuracy = 100 * correct / total

    model.train()  # Make sure training is back on

    return loss, accuracy

In [ ]:
def train_model(device, model, start_epoch, total_epochs, trainloader, validationloader, criterion, optimizer):
    model.to(device)
    
    print_every = 10
    steps = 0
    running_loss = 0
    start = time.time()
    
    for e in range(start_epoch, total_epochs):
        model.train()
        for images, labels in iter(trainloader):

            steps += 1
            # Move input and label tensors to the right device
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward and backward passes
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                valid_loss, valid_accuracy = get_metrics(device, model, validationloader, criterion)

                print("Epoch: {}/{}.. ".format(e + 1, total_epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss / print_every),
                      "Validation Loss: {:.3f}.. ".format(valid_loss / len(validationloader)),
                      "Validation Accuracy: {:.2f} %".format(valid_accuracy))

                running_loss = 0
    
    end = time.time()
    print('Training finished ! Time taken for whole training is {:.2f} seconds'.format(end - start))
    return e

In [ ]:
# PARAMETERS
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Enabling GPU if possible

# Let's choose a model (cf. https://cv-tricks.com/cnn/understand-resnet-alexnet-vgg-inception/)
model_name = 'vgg16' 
learning_rate = 0.001
total_epochs = 2

In [ ]:
# Build the model, the loss and the optimizer
# Since the model's forward method returns the log-softmax, use the negative log loss as criterion
model = build_model(model_name)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

In [ ]:
start_epoch = 0
epochs_done = train_model(device, model, start_epoch, total_epochs, train_dataloader, valid_dataloader, criterion, optimizer)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# Validation phase on the test set
_, accuracy = get_metrics(device, model, test_dataloader)
print("Test dataset accuracy: {:.2f} %".format(accuracy))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Save the checkpoint
def save_checkpoint(filename='checkpoint.pth'):    
    torch.save(checkpoint, filename)

In [ ]:
# Save the mapping of classes to indices got from one of the image datasets
model.class_to_idx = train_data.class_to_idx
checkpoint = {
    'nb_epochs_done': epochs_done + 1,
    'learning_rate': learning_rate,
    'model_name': model_name,
    'state_dict': model.state_dict()

}
# save_checkpoint() # commented to avoid mistakes and override of the already saved model

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# Function that loads a checkpoint and rebuilds the model
def load_checkpoint(filename):
    # Based on this discussion (https://discuss.pytorch.org/t/problem-loading-model-trained-on-gpu/17745/3) I had to add
    # map_location option to be able to load a model trained on GPU to then use it on CPU
    checkpoint = torch.load(filename, map_location=lambda storage, loc: storage)
    nb_epochs_done = checkpoint['nb_epochs_done']
    model = build_model(checkpoint['model_name'])
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model, nb_epochs_done

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    new_size = 224

    im = Image.open(image)
    # Resize but keeping aspect ratio
    im.thumbnail((256, 256))

    # Center crop to 224
    width, height = im.size  # Get new dimensions
    left = int(np.ceil((width - new_size) / 2))
    right = width - int(np.floor((width - new_size) / 2))
    top = int(np.ceil((height - new_size) / 2))
    bottom = height - int(np.floor((height - new_size) / 2))
    im = im.crop((left, top, right, bottom))

    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    # Scale the raw pixel intensities to the range [0, 1]
    np_image = np.array(im, dtype="float") / 255.0
    # Colors normalization
    np_image = (np_image - mean) / std

    return np_image.transpose((2, 0, 1))

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    input_img = process_image(image_path)
    torch_input_img = torch.from_numpy(input_img).float()
    torch_input_img.unsqueeze_(0) # Add a dimension at first position (will represent the batch size, here 1)
    
    model.eval()
    # Turn off gradients for validation --> saves memory and computations
    with torch.no_grad():
        output = model.forward(torch_input_img)
        probs, indices = output.topk(topk)
        # Transform tensors to numpy arrays and take the first (and single) element
        probs = np.exp(probs.numpy()[0]) # Do not forget to get back the exponential value as output is log-softmax !
        indices = indices.numpy()[0]
        # Revert the dict 'class to indice' to get 'indice to class'
        idx_classes = {v: k for k, v in train_data.class_to_idx.items()}
        classes = [v for k, v in idx_classes.items() if k in indices]
    
    return probs, classes

In [ ]:
# Load the trained & saved model
model_loaded, nb_epochs_done = load_checkpoint('checkpoint.pth')

In [ ]:
# Check with one image and print probs + classes
image_path = valid_dir + '/28/image_05267.jpg'
probs, classes = predict(image_path, model_loaded)
probs, classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
classnames = [cat_to_name[i] for i in classes]
print(classnames)

In [ ]:
# Display an image along with the top 5 classes
most_probable_classname = cat_to_name[classes[0]]
classnames = [cat_to_name[i] for i in classes]

figure, axis = plt.subplots(2, 1, figsize=(15, 10))
axis[0].set_title(most_probable_classname)
axis[0].set_axis_off()
axis[1].barh(np.arange(len(probs)), probs, tick_label=classnames)
axis[1].set_aspect(0.1)
axis[1].invert_yaxis()

imshow(torch.from_numpy(process_image(image_path)), axis[0])